In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
!pip install surprise

url = 'https://drive.google.com/file/d/1S0CtDB8NYUs94KgO0VDv6b2R1CShQcLF/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
links = pd.read_csv(path)


url = 'https://drive.google.com/file/d/1sW3zww6gMzoln0-U0Zs7HW_bKYjtH99i/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
movies = pd.read_csv(path)

url = 'https://drive.google.com/file/d/1nUpoWkhzhnYtUFvGYTR317RHiq7XtTx9/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
ratings = pd.read_csv(path)

url = 'https://drive.google.com/file/d/1F9szBIzHvE9sk-p89sk1zpxVEG_gJezg/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
tags = pd.read_csv(path)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=3193628 sha256=54667fcdd1e056b17a9560b97fb613ffbb02a9449ecb97ec15c94d14c1f9e7a9
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


### 1. Popularity_ranking_movies

It’s the most simple recommendations and non-personal. It based on the ratings, and always showed as the top n popular products, top 5 most sold items and so on. In this case, the rating refer to the quantity and quality. We use( quantity *quality) or set a threshold of quantity to implement a hybrid method.

In [2]:
tags.head(6)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
5,2,89774,Tom Hardy,1445715205


In [3]:
ratings.head(6)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400


In [4]:
links.head(5)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [5]:
movies.head(6)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller


In [6]:
# sort the ratings from high to low
ratings.sort_values('rating',ascending = False).head(5)

,userId,movieId,rating,timestamp
34031,232,3147,5.0,1218164903
50717,328,1080,5.0,1494210450
18166,115,487,5.0,944951378
18165,115,480,5.0,944953733
76157,479,3308,5.0,1039362558


In [7]:
#get the rating count for each movie #as for each movie there are mostly more than one userId rated it. so we should get the mean value of rating.
# rating_score can not added to DaraFrame ratings, because it less than the ratings
rating_score = pd.DataFrame(ratings.groupby('movieId').rating.mean()) #create a new DataFrame
rating_score['rating_count'] = ratings.groupby('movieId').rating.count()#for each movieId, count how many uses rated

In [8]:
rating_score = rating_score.merge(movies[['movieId','title']], on = 'movieId', how = 'left')#get the movie's title
rating_score['overall_rating'] = rating_score['rating']*(rating_score['rating_count']*0.1)

In [9]:
rating_score.sort_values('overall_rating',ascending = False).head(5)

,movieId,rating,rating_count,title,overall_rating
277,318,4.429022,317,"Shawshank Redemption, The (1994)",140.40
314,356,4.164134,329,Forrest Gump (1994),137.00
257,296,4.197068,307,Pulp Fiction (1994),128.85
1938,2571,4.192446,278,"Matrix, The (1999)",116.55
510,593,4.161290,279,"Silence of the Lambs, The (1991)",116.10


**### 2. Memory-based collaborative filtering**

In the collaborative filtering, we mainly use two methods of similarity calculating: Pearson and Cosine.

## 2.1 Item based

I followed the following steps>

• Create the big users-items table (pd.pivot_table())

• Compute Pearson similarities to target item

• Drop the NaNs (dropna)

• Add the rating count column

• Set the threshold of rating count

• Find the top n in terms of similarity of target item

In [10]:
# Create the big suders-items table
users_items = pd.pivot_table(ratings, index = 'userId', columns = 'movieId', values = 'rating')
users_items.head(5)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
target_movie = users_items[1] #select colum 1, movieId = 1, as the target movie
similarity = pd.DataFrame(users_items.corrwith(target_movie),columns = ['similarity']) # get the collaboration (Pearson) computing results.
similarity.dropna() #drop the NaNs
similarity = similarity.drop(labels = 1) #drop the target movie
similarity = similarity.merge(rating_score[['movieId','rating_count','title']], on = 'movieId', how = 'left')
similarity[similarity.rating_count > 10].sort_values('similarity',ascending = False).head(5)


,movieId,similarity,rating_count,title
9267,158238,0.968694,15,The Nice Guys (2016)
8674,122912,0.942264,13,Avengers: Infinity War - Part I (2018)
1316,1772,0.935897,12,Blues Brothers 2000 (1998)
2445,3261,0.922331,11,Singles (1992)
8430,112138,0.913282,19,22 Jump Street (2014)


In [12]:
#crate a function, output the most similar n movies to movie_name
def ralated_movie(movie_name, n):
  movie_ID = movies.loc[movies['title']==movie_name]['movieId'].tolist()[0] #get the movie_Id by the movie name
  movie_crosstab = pd.pivot_table(data = ratings, values='rating', index = 'userId', columns = 'movieId')
  similar_to_movie = pd.DataFrame(movie_crosstab.corrwith(movie_crosstab[movie_ID]),columns= ['collaboration'])#the corrwith results are Series type
  similar_to_movie.drop(movie_ID, inplace = True) 
  rating_count = pd.DataFrame({'rating_count':ratings.groupby('movieId')['rating'].count()})#.rename(columns = {'rating_count'})
  similar_to_movie = similar_to_movie.join(rating_count).dropna().reset_index()
  simelar_to_movie =  similar_to_movie.merge(movies[['movieId','title']], on = 'movieId', how = 'left')
  return simelar_to_movie.loc[similar_to_movie.rating_count >= 10].sort_values('collaboration',ascending = False).head(n)

ralated_movie('Forrest Gump (1994)',5)

,movieId,collaboration,rating_count,title
801,1295,0.932958,11,"Unbearable Lightness of Being, The (1988)"
3158,6793,0.885253,11,Beethoven (1992)
230,328,0.881682,10,Tales from the Crypt Presents: Demon Knight (1...
2640,4954,0.865633,11,Ocean's Eleven (a.k.a. Ocean's 11) (1960)
548,911,0.850591,13,Charade (1963)


In [13]:
def ralated_movie(movie_id, n):
  movie_crosstab = pd.pivot_table(data = ratings, values='rating', index = 'userId', columns = 'movieId')
  similar_to_movie = pd.DataFrame(movie_crosstab.corrwith(movie_crosstab[movie_id]),columns= ['collaboration']) #as the movie_crosstab is 
  similar_to_movie.drop(movie_id, inplace = True)
  rating_count = pd.DataFrame({'rating_count':ratings.groupby('movieId')['rating'].count()})#.rename(columns = {'rating_count'})
  similar_to_movie = similar_to_movie.join(rating_count).dropna().reset_index()
  simelar_to_movie =  similar_to_movie.merge(movies[['movieId','title']], on = 'movieId', how = 'left')
  return simelar_to_movie.loc[similar_to_movie.rating_count >= 10].sort_values('collaboration',ascending = False).head(n)

#simelar_to_movie.rating.mean()=17.4
ralated_movie(356,10)

,movieId,collaboration,rating_count,title
801,1295,0.932958,11,"Unbearable Lightness of Being, The (1988)"
3158,6793,0.885253,11,Beethoven (1992)
230,328,0.881682,10,Tales from the Crypt Presents: Demon Knight (1...
2640,4954,0.865633,11,Ocean's Eleven (a.k.a. Ocean's 11) (1960)
548,911,0.850591,13,Charade (1963)
4198,55721,0.799415,10,Elite Squad (Tropa de Elite) (2007)
136,195,0.786428,10,Something to Talk About (1995)
127,181,0.785661,17,Mighty Morphin Power Rangers: The Movie (1995)
4629,80906,0.782601,12,Inside Job (2010)
5327,150548,0.776636,10,Sherlock: The Abominable Bride (2016)


Instead of flitering out movie with a rating count below 10, let's consider a movie X as similar to Y only if at least 3 users have rated to both X and Y.

i.e. user 143, 153, and 168 watched both movies - not 3 random users watched X, and a different 3 random users watched y

In [14]:
def movie_simiar(movie_ID, n):
  movies_crosstab = pd.pivot_table(data=ratings, values='rating', index='userId', columns='movieId')#using pivot_talbe got the sparse matrix.
  coralation_rating = pd.DataFrame(movies_crosstab.corrwith(movies_crosstab[movie_ID]),columns = ['collaboration'])#got the coralation of restaurant_id with all other restaurant
  coralation_rating.drop(movie_ID,inplace = True)#drop the row of movie_ID itself
  rating_count = pd.DataFrame({'rating_count':ratings.groupby('movieId')['rating'].count()})#get the number of rating for each reataurant# (grouby+count) get a Series, so can not change the column name like 'place_crosstab"
  coralation_rating = coralation_rating.join(rating_count).dropna().reset_index() #add the rating_count to the coralation_rating
  coralation_rating = coralation_rating.merge(movies[['movieId','title']], on = 'movieId', how = 'left')#merge to DaraFrame geodata, get the name of restaurant
  coralation_rating = coralation_rating.sort_values('collaboration', ascending= False)# no restriction of rating count
  #coralation_rating = coralation_rating.loc[coralation_rating['rating_count']>=10].sort_values('collaboration', ascending= False) #setting the rating count to be bigger than 10
  movieId_list = coralation_rating.sort_values('collaboration', ascending= False)['movieId'].tolist()
  
  i_list=[]
  #for i in range(len(movieId_list)): it run too lang
  for i in range(100):# take the first 100 similar movies
    num = 0 # should be inside the forloob, otherwise, it will be accumulated
    for item in ratings.loc[ratings['movieId']==movie_ID]['userId'].tolist():
      if item in ratings.loc[ratings['movieId']==movieId_list[i]]['userId'].tolist():
        num=num+1
    if num>=3:
      i_list.append(movieId_list[i])
  return coralation_rating.loc[coralation_rating.movieId.isin(i_list)].sort_values('collaboration', ascending= False).head(n)

movie_simiar(1, 5)

,movieId,collaboration,rating_count,title
2268,4498,1.0,6,"Dead Pool, The (1988)"
237,358,1.0,9,Higher Learning (1995)
2687,5988,1.0,4,Quicksilver (1986)
296,459,1.0,6,"Getaway, The (1994)"
2416,4971,1.0,5,Moscow on the Hudson (1984)


## 2.2 User_based

The simple steps are :

• Create the big users-items table (pd.pivot_table())

• Replace NaNs with zeros (fillna(0, inplace =True)

• Compute pairwise cosine similarities

• Compute the weights.

• Find items user (001) has not rated.

• Compute the ratings user (001) would give to those unrated items.

• Find the top 5 items from the rating predictions.

In [15]:
#create a function to output the top n movies that user_ID could like
from sklearn.metrics.pairwise import cosine_similarity
from pandas.core.reshape.pivot import pivot_table


def recommendation_mov(user_ID, n):
  user_items = pivot_table(data = ratings,   #got the talbe that contains all the usage data, userId, rating and movieId
                           index = 'userId',
                           columns = 'movieId',
                           values = 'rating')
  user_items.fillna(0, inplace = True) #Fill the Na to 0
  not_watch_movies = user_items.loc[user_items.index!=user_ID,user_items.loc[user_ID,:]==0].T #got the movies that used_id did not rating
  user_similarities_moive = pd.DataFrame(cosine_similarity(user_items), #caculate the similarities
                                         index = user_items.index,
                                         columns = user_items.index)
  weight_movie = user_similarities_moive.loc[user_similarities_moive.index!=user_ID][user_ID]/sum(user_similarities_moive.loc[user_similarities_moive.index!=user_ID][user_ID]) #got the waight
  predicted_movie = pd.DataFrame(not_watch_movies.dot(weight_movie), columns = ['predicted_rating_movie']) #using weight to calculte the predict ratings
  predicted_movie = predicted_movie.merge(movies[['movieId','title']], on = 'movieId', how = 'left') #merge with data--movies, add the titles to movie
  return predicted_movie.sort_values('predicted_rating_movie', ascending = False).head(n) #sort the predicted_rating from high to low, and got the top n by 'head(n)'

recommendation_mov(1, 5)

,movieId,predicted_rating_movie,title
259,318,2.654727,"Shawshank Redemption, The (1994)"
475,589,2.087327,Terminator 2: Judgment Day (1991)
615,858,1.859548,"Godfather, The (1972)"
1895,2762,1.663564,"Sixth Sense, The (1999)"
3402,4993,1.624820,"Lord of the Rings: The Fellowship of the Ring,..."


### 3. Model based collaborative filtering

The model based collaborative filtering we learned based surprised, which is a Python scikit for building and analyzing recommender systems that deal with explicit rating data.

In [16]:
from surprise import SVD # ML model matrix facorization
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import GridSearchCV

In [17]:
reader = Reader(rating_scale=(1, 5)) #set the reader
data = ratings.drop(columns = ['timestamp']) #drop the column do not use
data_movie = Dataset.load_from_df(data, reader)
param_grid = {
    "n_epochs": [5, 10],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.4, 0.6]
}
gs_movie = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3,refit = True)

gs_movie.fit(data_movie)

print(gs_movie.best_score["rmse"])
print(gs_movie.best_params["rmse"])

0.8941812998511333
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [18]:
movies_crosstab = pd.pivot_table(ratings, index = 'userId', columns = 'movieId', values = 'rating')

In [19]:
#get the Top n movies of any users

def user_top_movie(userID, n):
  prediction =  []
  movieID = [] 
  for i in movies_crosstab.columns: # get all the movieId
    if movies_crosstab.isnull().loc[userID,i]: #select the movie the userId did not watch
      prediction.append(gs_movie.predict(userID, i).est) #get the predition 
      movieID.append(i)
    else:
      continue
  df_t = pd.DataFrame({'movieId':movieID, 'pre_rating':prediction})
  df_t= df_t.merge(movies[['movieId', 'title']],on='movieId',how = 'left')
  return df_t.sort_values('pre_rating', ascending = False).head(n)

user_top_movie(1,5)

,movieId,pre_rating,title
259,318,4.735360,"Shawshank Redemption, The (1994)"
560,750,4.684692,Dr. Strangelove or: How I Learned to Stop Worr...
615,858,4.667865,"Godfather, The (1972)"
834,1204,4.658950,Lawrence of Arabia (1962)
642,904,4.647378,Rear Window (1954)


In [20]:
#KNNWithMeans 
from surprise import KNNWithMeans
from surprise import Dataset
from surprise.model_selection import GridSearchCV

reader = Reader(rating_scale=(1, 5))

data_movie = Dataset.load_from_df(ratings[["userId", "movieId", "rating"]], reader)
sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4],
    "user_based": [False, True]
}

param_grid = {"sim_options": sim_options}

gs_knn_movie = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3,refit = True)

gs_knn_movie.fit(data_movie)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done c

In [21]:
#function based on KNNWithMeans
def user_top_movie_knn(userID, n):
  prediction =  []
  movieID = [] 
  for i in movies_crosstab.columns:
    if movies_crosstab.isnull().loc[userID,i]:
      prediction.append(gs_knn_movie.predict(userID, i).est)
      movieID.append(i)
  df_t = pd.DataFrame({'movieId':movieID, 'pre_rating':prediction})
  df_t = df_t.merge(movies[['movieId', 'title']],on='movieId',how = 'left')
  return df_t.sort_values('pre_rating', ascending = False).head(n)

user_top_movie_knn(1,5)

,movieId,pre_rating,title
6458,58425,5.0,Heima (2007)
5170,25805,5.0,"Atalante, L' (1934)"
2968,4323,5.0,"Horse Soldiers, The (1959)"
7957,103483,5.0,V/H/S/2 (2013)
7962,103602,5.0,Craig Ferguson: I'm Here To Help (2013)


In [22]:
#the second way of fillter the Null value
movies_crosstab.fillna(0, inplace =  True) # fill na=0, to select the null vaue in the following code
def user_top_movie(userID, n): #when using the 'movies_crosstab.loc[1,i]==0.0'as the null value select method, should fill na of the movies_crosstab to 0
  prediction =  []
  movieID = [] 
  for i in movies_crosstab.columns:
    if movies_crosstab.loc[userID,i]==0.0:
      prediction.append(gs_movie.predict(userID, i).est)
      movieID.append(i)
  df_t = pd.DataFrame({'movieId':movieID, 'pre_rating':prediction})
  df_t = df_t.merge(movies[['movieId', 'title']],on='movieId',how = 'left')
  return df_t.sort_values('pre_rating', ascending = False).head(n)

user_top_movie(1,5)

,movieId,pre_rating,title
259,318,4.735360,"Shawshank Redemption, The (1994)"
560,750,4.684692,Dr. Strangelove or: How I Learned to Stop Worr...
615,858,4.667865,"Godfather, The (1972)"
834,1204,4.658950,Lawrence of Arabia (1962)
642,904,4.647378,Rear Window (1954)
